## Import all the requiered libraries

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from hpelm import ELM
from sklearn.metrics import mean_squared_error




2024-08-30 21:51:06.374162: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-30 21:51:06.428399: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-30 21:51:06.429510: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-30 21:51:07.327799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load the dataset
data_m1 = pd.read_excel('data/M1.xlsx')

# Features (X) and target (y)
X_m1 = data_m1.drop('WPR', axis=1)
y_m1 = data_m1['WPR']

# Show the first 20 rows of the data
print(data_m1.head(20))

# Split data into training and testing sets
X_train_m1, X_test_m1, y_train_m1, y_test_m1 = train_test_split(X_m1, y_m1, test_size=0.2, random_state=42)

print("X_train_m1 shape:", X_train_m1.shape)
print("y_train_m1 shape:", y_train_m1.shape)


       Qa     Qc     Qd   Qhx   Ir     Qe       WPR
0   0.000  0.000  0.000  0.00    0  0.000  0.000000
1   0.000  0.000  0.000  0.00    0  0.000  0.000000
2   0.000  0.000  0.000  0.00    0  0.000  0.000000
3   0.000  0.000  0.000  0.00    0  0.000  0.000000
4   0.000  0.000  0.000  0.00    0  0.000  0.000000
5   0.000  0.000  0.000  0.00    0  0.000  0.000000
6   0.000  0.000  0.000  0.00    6  0.000  0.000000
7   0.000  0.000  0.000  0.00  122  0.000  0.000000
8   2.601  1.889  2.645  1.24  333  1.845  0.100440
9   5.770  4.702  5.911  1.70  517  4.560  1.051920
10  7.868  6.516  8.093  1.99  636  6.292  1.505088
11  8.792  7.302  9.057  2.11  670  7.037  1.938276
12  9.446  7.855  9.742  2.19  695  7.560  2.405268
13  8.669  7.198  8.928  2.09  634  6.938  2.019600
14  6.688  5.500  6.864  1.83  497  5.324  0.753840
15  3.562  2.751  3.632  1.39  302  2.681  0.000000
16  0.000  0.000  0.000  0.00   91  0.000  0.000000
17  0.000  0.000  0.000  0.00    2  0.000  0.000000
18  0.000  0

In [3]:
#show the first 20 rows of the data
print(data_m1.head(20))

       Qa     Qc     Qd   Qhx   Ir     Qe       WPR
0   0.000  0.000  0.000  0.00    0  0.000  0.000000
1   0.000  0.000  0.000  0.00    0  0.000  0.000000
2   0.000  0.000  0.000  0.00    0  0.000  0.000000
3   0.000  0.000  0.000  0.00    0  0.000  0.000000
4   0.000  0.000  0.000  0.00    0  0.000  0.000000
5   0.000  0.000  0.000  0.00    0  0.000  0.000000
6   0.000  0.000  0.000  0.00    6  0.000  0.000000
7   0.000  0.000  0.000  0.00  122  0.000  0.000000
8   2.601  1.889  2.645  1.24  333  1.845  0.100440
9   5.770  4.702  5.911  1.70  517  4.560  1.051920
10  7.868  6.516  8.093  1.99  636  6.292  1.505088
11  8.792  7.302  9.057  2.11  670  7.037  1.938276
12  9.446  7.855  9.742  2.19  695  7.560  2.405268
13  8.669  7.198  8.928  2.09  634  6.938  2.019600
14  6.688  5.500  6.864  1.83  497  5.324  0.753840
15  3.562  2.751  3.632  1.39  302  2.681  0.000000
16  0.000  0.000  0.000  0.00   91  0.000  0.000000
17  0.000  0.000  0.000  0.00    2  0.000  0.000000
18  0.000  0

In [4]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Initialize XGBoost model
xgb_model = XGBRegressor()

# Fit the model on the training data
xgb_model.fit(X_train_m1, y_train_m1)

# Predict on the test data
y_pred_xgb = xgb_model.predict(X_test_m1)

# Evaluate the model
mse_xgb = mean_squared_error(y_test_m1, y_pred_xgb)
print(f'XGBoost MSE: {mse_xgb}')


XGBoost MSE: 0.8034212536739505


In [5]:
# Initialize ELM model
elm_model = ELM(X_train_m1.shape[1], 1)

# Add hidden neurons (for example, 100 neurons)
elm_model.add_neurons(100, 'sigm')

# Train the ELM model
#elm_model.train(X_train_m1, y_train_m1.values.reshape(-1, 1), 'r')

# Predict on the test data
#y_pred_elm = elm_model.predict(X_test_m1)

# Evaluate the model
#mse_elm = mean_squared_error(y_test_m1, y_pred_elm)
#print(f'ELM MSE: {mse_elm}')


In [6]:
from sklearn.ensemble import RandomForestRegressor

# Initialize Random Forest model
rf_model = RandomForestRegressor()

# Fit the model on the training data
rf_model.fit(X_train_m1, y_train_m1)

# Predict on the test data
y_pred_rf = rf_model.predict(X_test_m1)

# Evaluate the model
mse_rf = mean_squared_error(y_test_m1, y_pred_rf)
print(f'Random Forest MSE: {mse_rf}')


Random Forest MSE: 0.7140468277221624


In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Reshape data for LSTM (samples, timesteps, features)
X_train_lstm = X_train_m1.values.reshape((X_train_m1.shape[0], 1, X_train_m1.shape[1]))
X_test_lstm = X_test_m1.values.reshape((X_test_m1.shape[0], 1, X_test_m1.shape[1]))

# Initialize LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

# Fit the model
lstm_model.fit(X_train_lstm, y_train_m1, epochs=100, verbose=1)

# Predict on the test data
y_pred_lstm = lstm_model.predict(X_test_lstm)

# Evaluate the model
mse_lstm = mean_squared_error(y_test_m1, y_pred_lstm)
print(f'LSTM MSE: {mse_lstm}')


2024-08-30 21:51:12.315024: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/100
219/219 [==============================] - 1s 2ms/step - loss: 2.4540
Epoch 2/100
219/219 [==============================] - 0s 2ms/step - loss: 0.7301
Epoch 3/100
219/219 [==============================] - 0s 2ms/step - loss: 0.7270
Epoch 4/100
219/219 [==============================] - 0s 2ms/step - loss: 0.7313
Epoch 5/100
219/219 [==============================] - 0s 2ms/step - loss: 0.7352
Epoch 6/100
219/219 [==============================] - 1s 2ms/step - loss: 0.7284
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 0.7361
Epoch 8/100
219/219 [==============================] - 0s 2ms/step - loss: 0.7335
Epoch 9/100
219/219 [==============================] - 0s 2ms/step - loss: 0.7240
Epoch 10/100
219/219 [==============================] - 0s 2ms/step - loss: 0.7349
Epoch 11/100
219/219 [==============================] - 0s 2ms/step - loss: 0.7255
Epoch 12/100
219/219 [==============================] - 0s 2ms/step - loss: 0.7308
Epoch 13/100


In [8]:
#PSO optimization for XGBoost
from pyswarm import pso

def objective_function(params):
    xgb_model = XGBRegressor(learning_rate=params[0], max_depth=int(params[1]), n_estimators=int(params[2]))
    xgb_model.fit(X_train_m1, y_train_m1)
    y_pred = xgb_model.predict(X_test_m1)
    return mean_squared_error(y_test_m1, y_pred)

# Define the bounds for learning_rate, max_depth, and n_estimators
lb = [0.01, 3, 50]
ub = [0.3, 10, 300]

# Run PSO
opt_params, mse_pso = pso(objective_function, lb, ub)
print(f'PSO optimized MSE: {mse_pso}, with parameters: {opt_params}')


Stopping search: Swarm best objective change less than 1e-08
PSO optimized MSE: 0.5841977987641966, with parameters: [7.34752025e-02 3.48308627e+00 1.23978010e+02]


In [13]:
#PSO for Random Forest
def objective_function(params):
    n_estimators, max_depth = int(params[0]), int(params[1])
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train_m1, y_train_m1)
    predictions = model.predict(X_test_m1)
    accuracy = accuracy_score(np.argmax(y_test_m1, axis=1), np.argmax(predictions, axis=1))
    return 1 - accuracy  # Minimize error (maximize accuracy)


In [14]:
# Define bounds for hyperparameters
lb = [10, 5]   # Lower bounds for n_estimators and max_depth
ub = [200, 50] # Upper bounds for n_estimators and max_depth

# Run PSO
best_params, best_score = pso(objective_function, lb, ub, swarmsize=20, maxiter=100)
print(f'Best parameters: {best_params}, Best score: {1 - best_score}')


ValueError: `axis` must be fewer than the number of dimensions (1)

In [9]:
import random
from deap import base, creator, tools, algorithms
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# Define the fitness function
def fitness_function(individual):
    xgb_model = XGBRegressor(learning_rate=individual[0], max_depth=int(individual[1]), n_estimators=int(individual[2]))
    xgb_model.fit(X_train_m1, y_train_m1)
    y_pred = xgb_model.predict(X_test_m1)
    return mean_squared_error(y_test_m1, y_pred),

# Create types
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Register functions
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0.01, 0.3)
toolbox.register("attr_int", random.randint, 3, 10)
toolbox.register("attr_int2", random.randint, 50, 300)
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_float, toolbox.attr_int, toolbox.attr_int2), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxBlend, alpha=0.5)

# Mutation function that constrains the learning_rate to [0.01, 0.3]
def constrained_mutation(individual, indpb):
    if random.random() < indpb:
        individual[0] = min(max(individual[0] + random.gauss(0, 0.05), 0.01), 0.3)
    if random.random() < indpb:
        individual[1] = random.randint(3, 10)
    if random.random() < indpb:
        individual[2] = random.randint(50, 300)
    return individual,

toolbox.register("mutate", constrained_mutation, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Define population and evolution
population = toolbox.population(n=10)
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=40, verbose=True)

# Extract best individual
best_individual = tools.selBest(population, 1)[0]
print(f'GA optimized parameters: {best_individual}')


gen	nevals
0  	10    
1  	7     
2  	9     
3  	6     
4  	9     
5  	8     
6  	6     
7  	6     
8  	9     
9  	8     
10 	6     
11 	10    
12 	7     
13 	7     
14 	10    
15 	7     
16 	8     
17 	8     
18 	9     
19 	10    
20 	6     
21 	10    
22 	8     
23 	6     
24 	8     
25 	8     
26 	8     
27 	9     
28 	3     
29 	10    
30 	9     
31 	8     
32 	6     
33 	6     
34 	8     
35 	10    
36 	10    
37 	7     
38 	10    
39 	8     
40 	8     
GA optimized parameters: [0.04370964868107195, 3.4376864695206732, 84.0]
